In [2]:
# Installing the necessary libraries

!pip install nsepy
!pip install pandas
!pip install numpy
!pip install TA-Lib
!pip install plotly
!pip install yfinance

In [3]:
# Organizing the imports 

from nsepy import get_history
from datetime import date
import pandas as pd
import numpy as np
import talib
import datetime
import yfinance as yf
import plotly.graph_objects as go


In [4]:
# # Plot candles

# def plot_candle(df, title):
#     candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])

#     # Plot only the last month's data
#     fig = go.Figure(data=[candlestick],
#                     layout=go.Layout(title=go.layout.Title(text=title)))
#     fig.layout.xaxis.type = 'category' 
#     fig.show()


# stock = fetch_data('INFY')

In [5]:
# # Printing the data
# scrip_name = 'INFY'
# stock_df = fetch_data(scrip_name)

# candlestick = go.Candlestick(x=stock_df.index, 
#                              open=stock_df['Open'],  
#                              high=stock_df['High'], 
#                              low=stock_df['Low'], 
#                              close=stock_df['Close'])

# fig = go.Figure(data=[candlestick])
# fig.layout.xaxis.type = 'category' 
# fig.show()

In [6]:
# Fetching data from yfinance
 
def fetch_data(scrip_name):
    start_date = date(2023, 1, 1) - datetime.timedelta(300)
    return yf.Ticker(scrip_name).history(
                    start=start_date, 
                    end=date(2023,9,10))



In [7]:
def get_macd_indicator(row):
    last_hist = row['hist']
    prev_hist = row['prev_hist']
    indicator = 'HOLD'
    # Provide BUY or SELL indications during crossovers
    if not np.isnan(prev_hist) and not np.isnan(last_hist):
        # If hist value has changed from negative to positive or vice versa, it indicates a crossover
        macd_crossover = (abs(last_hist + prev_hist)) != (abs(last_hist) + abs(prev_hist))
        if macd_crossover:
            indicator = 'BUY' if last_hist > 0 else 'SELL'
            
    return indicator


def apply_macd_strategy(stock_df):
    stock_df['macd'],  stock_df['signal'],  stock_df['hist'] = talib.MACD(stock_df['Close'], 
                                fastperiod=12, slowperiod=26, signalperiod=9)
    stock_df['prev_hist'] = stock_df['hist'].shift(1)
    stock_df['macd_indicator'] = stock_df.apply(lambda row: get_macd_indicator(row), axis=1)
    
    return stock_df


def apply_engulfing_pattern(stock_df):
    stock_df['engulf'] = talib.CDLENGULFING(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])
    stock_df['engulf_indicator'] = stock_df.apply(lambda row: 'BUY' if (row['engulf'] > 0) else 'SELL' if (row['engulf'] < 0) else 'HOLD', axis=1)
    return stock_df

# BUY/SELL/HOLD indicator based on the 200-days, 50-days and 20-days simple moving average
def get_ma_indicator(row):
    indicator = 'HOLD'
    row_values = [row['Close'], row['20_sma'], row['50_sma'] , row['200_sma']]
    
    # If close < 20-day sma < 50-day sma < 200-day sma --> BUY
    # If close > 20-day sma > 50-day sma > 200-day sma --> SELL
    if(row_values == sorted(row_values)):
        indicator = 'BUY'
    elif (row_values == sorted(row_values, reverse=True)):
        indicator = 'SELL'
        
    return indicator

def apply_moving_average_strategy(stock_df):
    # Compute the moving averages for 20 days, 50 days and 200 days
    stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_sma'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['200_sma'] = stock_df['Close'].rolling(window=200).mean()

    stock_df['MA_indicator'] = stock_df.apply(lambda row: get_ma_indicator(row), axis=1)
    return stock_df

# Compute RSI
def apply_rsi_strategy(stock_df, rsi_period = 14):
    stock_df['rsi'] = talib.RSI(stock_df['Close'], rsi_period) 
    return stock_df

In [62]:
# Show the necessary - BUY/SELL/HOLD signals

result = []

def show_signals(scrip_name, stock):
    buy_count = 0
    sell_count = 0
    bought = False
    for index, row in stock.iterrows(): 
        # print(index)
        if (row['indicator'] == 'BUY') and index.date() >= date(2023, 9, 1):
            print(row)
            bought = True
            buy_count += 1
        if (row['indicator'] == 'SELL') and index.date() >= date(2023, 9, 1):
            print(row)
            bought = False
            sell_count += 1
    # print("buy count:", buy_count, "sell count:", sell_count)
    

In [9]:
CHECK_RSI_LAST_SESSIONS = 3
RSI_LOWER_LIMIT = 40
RSI_UPPER_LIMIT = 60
def apply_trading_algo(stock_df):
    stock_df['indicator'] = 'HOLD'
    buy_sell_signals = stock_df[(stock_df['macd_indicator'] != 'HOLD') | (stock_df['MA_indicator'] != 'HOLD')| (stock_df['engulf_indicator'] != 'HOLD')]
    for index,row in buy_sell_signals.iterrows():
        # Get previous n sessions whenever a signal is reached 
        analysis_df = stock_df.loc[:index].tail(3)
        macd_ind = analysis_df.iloc[-1]['macd_indicator']
        ma_ind = analysis_df.iloc[-1]['MA_indicator']
        engulf_ind = analysis_df.iloc[-1]['engulf_indicator']
        current_indicator = 'HOLD' 
        
        # Preference given in this order: MACD, MA, engulfing pattern
        current_indicator = macd_ind if macd_ind != 'HOLD' else \
                            ma_ind if ma_ind != 'HOLD' else engulf_ind
        
        # Decide the final indicator based on the RSI value
        if ((current_indicator == 'BUY') and (analysis_df['rsi'].min() <= RSI_LOWER_LIMIT)) \
            or \
            ((current_indicator == 'SELL') and (analysis_df['rsi'].max() >= RSI_UPPER_LIMIT)): 
            stock_df.at[index, 'indicator'] = current_indicator
        
    return stock_df



In [10]:
# The main method to determine the signals


def my_trading_algo(scrip_name):
    # Fetch the NSE data
    stock = fetch_data(scrip_name)

    # Apply the strategies individual
    stock = apply_moving_average_strategy(stock)
    stock = apply_rsi_strategy(stock)
    stock = apply_macd_strategy(stock)
    stock = apply_engulfing_pattern(stock)

    # Get only 2021 data for further analysis
    # import pdb; pdb.set_trace()
    stock = stock[stock.index >= str(date(2023,1,1))]

    # stock = print(stock)

    # Merge the strategies to create a final BUY/SELL/HOLD indicator
    stock = apply_trading_algo(stock)

    # # Plot the buy/sell recommendations on 2021 data
    show_signals(scrip_name, stock)
    
    return stock

In [65]:
data = my_trading_algo("RELIANCE.NS")

Open                     2440.0
High                     2456.0
Low                 2422.949951
Close               2448.199951
Volume                 11554644
Dividends                   0.0
Stock Splits                0.0
20_sma              2475.203174
50_sma              2547.354297
200_sma             2468.278363
MA_indicator               HOLD
rsi                   43.241299
macd                 -31.450125
signal               -31.923372
hist                   0.473247
prev_hist             -2.372015
macd_indicator              BUY
engulf                        0
engulf_indicator           HOLD
indicator                   BUY
Name: 2023-09-08 00:00:00+05:30, dtype: object
